# Finetuning Ranker

# Installs

In [1]:
pip install bitsandbytes

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
pip install accelerate -U

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
!pip install transformers[torch]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Infra

## Paths

In [6]:
PATH_TRAIN_MODEL_LOCAL = "/content/drive/MyDrive/treinamento/202301_IA368DD/indir/train/monot5"

## Função de verificação de memória

In [7]:
from psutil import virtual_memory

In [8]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Jun 26 01:18:44 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   67C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [9]:
def mostra_memoria(lista_mem=['cpu']):
  """
  Esta função exibe informações de memória da CPU e/ou GPU, conforme parâmetros fornecidos.

  Parâmetros:
  -----------
  lista_mem : list, opcional
      Lista com strings 'cpu' e/ou 'gpu'.
      'cpu' - exibe informações de memória da CPU.
      'gpu' - exibe informações de memória da GPU (se disponível).
      O valor padrão é ['cpu'].

  Saída:
  -------
  A função não retorna nada, apenas exibe as informações na tela.

  Exemplo de uso:
  ---------------
  Para exibir informações de memória da CPU:
      mostra_memoria(['cpu'])

  Para exibir informações de memória da CPU e GPU:
      mostra_memoria(['cpu', 'gpu'])

  Autor: Marcus Vinícius Borela de Castro

  """
  if 'cpu' in lista_mem:
    vm = virtual_memory()
    ram={}
    ram['total']=round(vm.total / 1e9,2)
    ram['available']=round(virtual_memory().available / 1e9,2)
    # ram['percent']=round(virtual_memory().percent / 1e9,2)
    ram['used']=round(virtual_memory().used / 1e9,2)
    ram['free']=round(virtual_memory().free / 1e9,2)
    ram['active']=round(virtual_memory().active / 1e9,2)
    ram['inactive']=round(virtual_memory().inactive / 1e9,2)
    ram['buffers']=round(virtual_memory().buffers / 1e9,2)
    ram['cached']=round(virtual_memory().cached/1e9 ,2)
    print(f"Your runtime RAM in gb: \n total {ram['total']}\n available {ram['available']}\n used {ram['used']}\n free {ram['free']}\n cached {ram['cached']}\n buffers {ram['buffers']}")
    print('/nGPU')
    gpu_info = !nvidia-smi
  if 'gpu' in lista_mem:
    gpu_info = '\n'.join(gpu_info)
    if gpu_info.find('failed') >= 0:
      print('Not connected to a GPU')
    else:
      print(gpu_info)


In [10]:
mostra_memoria(['cpu','gpu'])

Your runtime RAM in gb: 
 total 13.61
 available 12.45
 used 0.83
 free 6.54
 cached 6.19
 buffers 0.06
/nGPU
Mon Jun 26 01:18:44 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   67C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+--

## Imports

In [11]:
import os

In [12]:
from transformers import Trainer
import numpy as np
import torch


In [13]:
import pandas as pd

In [14]:
from dataclasses import dataclass, field

In [15]:
from datasets import Dataset

In [16]:
import json
from transformers import (
    AutoTokenizer,
    AutoConfig,
    MT5Tokenizer,
    AutoModelForSeq2SeqLM,
    AutoModelForSequenceClassification,
    Trainer,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    DataCollatorWithPadding,
    DataCollatorForSeq2Seq,
)
from tqdm.auto import tqdm
from datasets import Dataset
from dataclasses import dataclass, field


## Preparando para debug e display

In [17]:
def config_display():
  """
  Esta função configura as opções de display do Pandas.
  """

  # Configurando formato saída Pandas
  # define o número máximo de colunas que serão exibidas
  pd.options.display.max_columns = None

  # define a largura máxima de uma linha
  pd.options.display.width = 1000

  # define o número máximo de linhas que serão exibidas
  pd.options.display.max_rows = 100

  # define o número máximo de caracteres por coluna
  pd.options.display.max_colwidth = 50

  # se deve exibir o número de linhas e colunas de um DataFrame.
  pd.options.display.show_dimensions = True

  # número de dígitos após a vírgula decimal a serem exibidos para floats.
  pd.options.display.precision = 7


In [18]:
def config_debug():
  """
  Esta função configura as opções de debug do PyTorch e dos pacotes
  transformers e datasets.
  """

  # Define opções de impressão de tensores para o modo científico
  torch.set_printoptions(sci_mode=True)
  """
    Significa que valores muito grandes ou muito pequenos são mostrados em notação científica.
    Por exemplo, em vez de imprimir o número 0.0000012345 como 0.0000012345,
    ele seria impresso como 1.2345e-06. Isso é útil em situações em que os valores dos tensores
    envolvidos nas operações são muito grandes ou pequenos, e a notação científica permite
    uma melhor compreensão dos números envolvidos.
  """

  # Habilita detecção de anomalias no autograd do PyTorch
  torch.autograd.set_detect_anomaly(True)
  """
    Permite identificar operações que podem causar problemas de estabilidade numérica,
    como gradientes explodindo ou desaparecendo. Quando essa opção é ativada,
    o PyTorch verifica se há operações que geram valores NaN ou infinitos nos tensores
    envolvidos no cálculo do gradiente. Se for detectado um valor anômalo, o PyTorch
    interrompe a execução e gera uma exceção, permitindo que o erro seja corrigido
    antes que se torne um problema maior.

    É importante notar que a detecção de anomalias pode ter um impacto significativo
    no desempenho, especialmente em modelos grandes e complexos. Por esse motivo,
    ela deve ser usada com cautela e apenas para depuração.
  """

  # Configura variável de ambiente para habilitar a execução síncrona (bloqueante) das chamadas da API do CUDA.
  os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
  """
    o Python aguarda o término da execução de uma chamada da API do CUDA antes de executar a próxima chamada.
    Isso é útil para depurar erros no código que envolve operações na GPU, pois permite que o erro seja capturado
    no momento em que ocorre, e não depois de uma sequência de operações que pode tornar a origem do erro mais difícil de determinar.
    No entanto, é importante lembrar que esse modo de execução é significativamente mais lento do que a execução assíncrona,
    que é o comportamento padrão do CUDA. Por isso, é recomendado utilizar esse comando apenas em situações de depuração
    e removê-lo após a solução do problema.
  """

  # Define o nível de verbosity do pacote transformers para info
  # transformers.utils.logging.set_verbosity_info()


  """
    Define o nível de detalhamento das mensagens de log geradas pela biblioteca Hugging Face Transformers
    para o nível info. Isso significa que a biblioteca irá imprimir mensagens de log informativas sobre
    o andamento da execução, tais como tempo de execução, tamanho de batches, etc.

    Essas informações podem ser úteis para entender o que está acontecendo durante a execução da tarefa
    e auxiliar no processo de debug. É importante notar que, em alguns casos, a quantidade de informações
    geradas pode ser muito grande, o que pode afetar o desempenho do sistema e dificultar a visualização
    das informações relevantes. Por isso, é importante ajustar o nível de detalhamento de acordo com a
    necessidade de cada tarefa.

    Caso queira reduzir a quantidade de mensagens, comentar a linha acima e
      descomentar as duas linhas abaixo, para definir o nível de verbosity como error ou warning

    transformers.utils.logging.set_verbosity_error()
    transformers.utils.logging.set_verbosity_warning()
  """


  # Define o modo verbose do xmode, que é utilizado no debug
  # %xmode Verbose

  """
    Comando usado no Jupyter Notebook para controlar o modo de exibição das informações de exceções.
    O modo verbose é um modo detalhado que exibe informações adicionais ao imprimir as exceções.
    Ele inclui as informações de pilha de chamadas completa e valores de variáveis locais e globais
    no momento da exceção. Isso pode ser útil para depurar e encontrar a causa de exceções em seu código.
    Ao usar %xmode Verbose, as informações de exceção serão impressas com mais detalhes e informações adicionais serão incluídas.

    Caso queira desabilitar o modo verbose e utilizar o modo plain,
    comentar a linha acima e descomentar a linha abaixo:
    %xmode Plain
  """

  """
    Dica:
    1.  pdb (Python Debugger)
      Quando ocorre uma exceção em uma parte do código, o programa para a execução e exibe uma mensagem de erro
      com informações sobre a exceção, como a linha do código em que ocorreu o erro e o tipo da exceção.

      Se você estiver depurando o código e quiser examinar o estado das variáveis ​​e executar outras operações
      no momento em que a exceção ocorreu, pode usar o pdb (Python Debugger). Para isso, é preciso colocar o comando %debug
      logo após ocorrer a exceção. Isso fará com que o programa pare na linha em que ocorreu a exceção e abra o pdb,
      permitindo que você explore o estado das variáveis, examine a pilha de chamadas e execute outras operações para depurar o código.


    2. ipdb
      O ipdb é um depurador interativo para o Python que oferece recursos mais avançados do que o pdb,
      incluindo a capacidade de navegar pelo código fonte enquanto depura.

      Você pode começar a depurar seu código inserindo o comando ipdb.set_trace() em qualquer lugar do
      seu código onde deseja pausar a execução e começar a depurar. Quando a execução chegar nessa linha,
      o depurador entrará em ação, permitindo que você examine o estado atual do seu programa e execute
      comandos para investigar o comportamento.

      Durante a depuração, você pode usar comandos:
        next (para executar a próxima linha de código),
        step (para entrar em uma função chamada na próxima linha de código)
        continue (para continuar a execução normalmente até o próximo ponto de interrupção).

      Ao contrário do pdb, o ipdb é um depurador interativo que permite navegar pelo código fonte em que
      está trabalhando enquanto depura, permitindo que você inspecione variáveis, defina pontos de interrupção
      adicionais e até mesmo execute expressões Python no contexto do seu programa.
  """


In [19]:
config_display()

In [20]:
config_debug()

## Vinculando pasta do google drive para salvar dados

In [21]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [22]:
current_dir = os.getcwd()
print("Current directory:", current_dir)

Current directory: /content


## Constants

In [23]:
TOKEN_FALSE = '▁não'
TOKEN_TRUE = '▁sim'

In [24]:
MODEL_NAME = 'unicamp-dl/ptt5-base-pt-msmarco-100k-v2'
# 'unicamp-dl/mt5-3B-mmarco-en-pt'

# Carga dos dados

In [25]:
PATH_LOCAL_DATA = '/content/drive/MyDrive/treinamento/202301_IA368DD/indir/data'

In [26]:
# path_data = '/content/drive/MyDrive/treinamento/202301_IA368DD/indir/data/train_data_juris_tcu_index_bm25.csv'
PATH_TRAIN_DATA_ZIP = f"{PATH_LOCAL_DATA}/train_data_juris_tcu_index_limit_50_per_doc.zip"
PATH_TRAIN_DATA = f"{PATH_LOCAL_DATA}/train_data_juris_tcu_index_limit_50_per_doc.csv"

In [27]:
os.path.exists(PATH_TRAIN_DATA)

True

In [ ]:
%%time
if not os.path.exists(PATH_TRAIN_DATA):
  import zipfile
  !wget https://github.com/marcusborela/ind-ir/raw/main/data/train_juris_tcu_index/train_data_juris_tcu_index_limit_{alt_aqui}_per_doc.zip -O {PATH_TRAIN_DATA_ZIP}

  # Extrair o arquivo zip
  with zipfile.ZipFile(PATH_TRAIN_DATA_ZIP, 'r') as zip_ref:
      zip_ref.extractall(PATH_LOCAL_DATA)

  # Listar os arquivos extraídos
  arquivos_extraidos = zip_ref.namelist()
  # Exibir os arquivos extraídos
  for arquivo in arquivos_extraidos:
      print(arquivo)
  print("File loaded")
else:
  print("File already there!")

In [28]:
df = pd.read_csv(PATH_TRAIN_DATA)

In [29]:
df.shape
# lim 100(111852, 6)

(86804, 6)

Verificando correção do arquivo!

In [30]:
print(df.isnull().sum())

QUERY_ID          0
DOC_ID            0
RELEVANCE         0
TYPE_RELEVANCE    0
DOC_TEXT          0
QUERY_TEXT        0
Length: 6, dtype: int64


In [31]:
df[['QUERY_TEXT','DOC_TEXT']].applymap(len).describe()

,QUERY_TEXT,DOC_TEXT
count,86804.0000000,86804.0000000
mean,332.0505967,849.8926778
std,174.7122769,522.1150133
min,41.0000000,86.0000000
25%,222.0000000,538.0000000
50%,301.0000000,744.0000000
75%,401.0000000,1066.0000000
max,4212.0000000,3739.0000000


Para cada positivo, tem 1 negativo

In [32]:
df['RELEVANCE'].describe()

count    86804.0000000
mean         0.5000000
std          0.5000029
min          0.0000000
25%          0.0000000
50%          0.5000000
75%          1.0000000
max          1.0000000
Name: RELEVANCE, Length: 8, dtype: float64

In [33]:
df.head()

,QUERY_ID,DOC_ID,RELEVANCE,TYPE_RELEVANCE,DOC_TEXT,QUERY_TEXT
0,151655,775,1,AREA,"O termo é ""Responsabilidade"".\nResponsabilidad...",O dever de observância à hierarquia militar nã...
1,151655,5095,0,AREA,"O termo é ""Competência do TCU"".\nCompetência d...",O dever de observância à hierarquia militar nã...
2,151655,1943,1,TEMA,"O termo é ""Agente público"".\nAgente público te...",O dever de observância à hierarquia militar nã...
3,151655,1322,0,TEMA,"O termo é ""Receita pública"".\nReceita pública ...",O dever de observância à hierarquia militar nã...
4,151655,15694,1,SUBTEMA,"O termo é ""Hierarquia"".\nHierarquia tem defini...",O dever de observância à hierarquia militar nã...


In [34]:
df["label"] = [TOKEN_FALSE if relevance == 0 else TOKEN_TRUE for relevance in df["RELEVANCE"]]

In [35]:
df.head()

,QUERY_ID,DOC_ID,RELEVANCE,TYPE_RELEVANCE,DOC_TEXT,QUERY_TEXT,label
0,151655,775,1,AREA,"O termo é ""Responsabilidade"".\nResponsabilidad...",O dever de observância à hierarquia militar nã...,▁sim
1,151655,5095,0,AREA,"O termo é ""Competência do TCU"".\nCompetência d...",O dever de observância à hierarquia militar nã...,▁não
2,151655,1943,1,TEMA,"O termo é ""Agente público"".\nAgente público te...",O dever de observância à hierarquia militar nã...,▁sim
3,151655,1322,0,TEMA,"O termo é ""Receita pública"".\nReceita pública ...",O dever de observância à hierarquia militar nã...,▁não
4,151655,15694,1,SUBTEMA,"O termo é ""Hierarquia"".\nHierarquia tem defini...",O dever de observância à hierarquia militar nã...,▁sim


In [36]:
df.rename(columns={'DOC_TEXT': 'text', 'QUERY_TEXT':'query'},inplace=True)

In [37]:
df.head()

,QUERY_ID,DOC_ID,RELEVANCE,TYPE_RELEVANCE,text,query,label
0,151655,775,1,AREA,"O termo é ""Responsabilidade"".\nResponsabilidad...",O dever de observância à hierarquia militar nã...,▁sim
1,151655,5095,0,AREA,"O termo é ""Competência do TCU"".\nCompetência d...",O dever de observância à hierarquia militar nã...,▁não
2,151655,1943,1,TEMA,"O termo é ""Agente público"".\nAgente público te...",O dever de observância à hierarquia militar nã...,▁sim
3,151655,1322,0,TEMA,"O termo é ""Receita pública"".\nReceita pública ...",O dever de observância à hierarquia militar nã...,▁não
4,151655,15694,1,SUBTEMA,"O termo é ""Hierarquia"".\nHierarquia tem defini...",O dever de observância à hierarquia militar nã...,▁sim


# Separating evaluation data and prepare dataset tokenized

In [38]:
from sklearn.model_selection import train_test_split

In [39]:
train_df, valid_df = train_test_split(df, test_size=0.01,
                                      stratify=df['label'].values, random_state=123)
# Definir os argumentos de treinamento

In [40]:
train_df.shape, valid_df.shape

((85935, 7), (869, 7))

In [41]:
valid_df[:2]

,QUERY_ID,DOC_ID,RELEVANCE,TYPE_RELEVANCE,text,query,label
76282,17906,1287,1,INDEXACAO_EXTRA,"O termo é ""Projeto executivo"".\nProjeto execut...",É lícita a utilização de pregão para contrataç...,▁sim
73875,18449,5095,0,INDEXACAO_EXTRA,"O termo é ""Competência do TCU"".\nCompetência d...",A Administração deve realizar as conformidades...,▁não


In [42]:
print(np.unique(train_df['label'], return_counts=True), '\n', np.unique(valid_df['label'], return_counts=True))

(array(['▁não', '▁sim'], dtype=object), array([42967, 42968])) 
 (array(['▁não', '▁sim'], dtype=object), array([435, 434]))


In [43]:
train_dataset = Dataset.from_pandas(train_df[["query", "text", "label"]].reset_index(drop=True))
valid_dataset = Dataset.from_pandas(valid_df[["query", "text", "label"]].reset_index(drop=True))

In [44]:
len(train_dataset), len(valid_dataset)

(85935, 869)

In [45]:
valid_dataset[0]

{'query': 'É lícita a utilização de pregão para contratação de serviço técnico de apoio à fiscalização de projetos executivos e de execução de obras de engenharia.',
 'text': 'O termo é "Projeto executivo".\nProjeto executivo tem definição: "Conjunto dos elementos necessários e suficientes à execução completa da obra, de acordo com as normas pertinentes da Associação Brasileira de Normas Técnicas (ABNT).".\nProjeto executivo é uma especialização de: "Projeto de engenharia".\nProjeto executivo tem termo relacionado: "Edital de licitação", "Execução de obras e serviços", "Memória de cálculo", "Licitação de melhor técnica", "Licitação de técnica e preço", "RDC", "Obra pública", "Contratação integrada", "Contratação semi-integrada", "Concorrência pública", "Convite (Licitação)", "Cronograma de desembolso", "Memorial descritivo", "Obras e serviços de engenharia", "Orçamento detalhado", "Prestação de serviço", "Responsabilidade técnica", "Tomada de preços", "Viabilidade econômica" e "Viabili

In [46]:
# tokenizer = MT5Tokenizer.from_pretrained(MODEL_NAME)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [47]:
def tokenize(batch):
    queries_documents = [f"Query: {query} Document: {text} Relevant:" for query, text in zip(batch["query"], batch["text"])]
    print(f"Chamado tokenize len(queries_documents): {len(queries_documents)}")
    tokenized = tokenizer(
        queries_documents,
        padding=True, # "max_length",
        truncation=True,
        # return_tensors="pt",
        max_length= 512
    )
    # tokenized["labels"] = [[label] for label in batch["label"]]
    # tokenized['label'] = [[token_false, token_true][int(pairs["label"][i])]
    tokenized["labels"] = tokenizer(batch['label'])['input_ids']
    # tokenized["labels"] = [tokenizer.get_vocab()[token] for token in batch['label']]
    # tokenized["labels"] = [token_id_true if label == 'true' else token_id_false for label in batch["label"]]
    return tokenized


In [48]:
%%time
# valid_dataset.set_transform(tokenize)
valid_dataset = valid_dataset.map(
        tokenize,
        remove_columns=('query', 'text', 'label'),
        batched=True,
        desc='Tokenizing',
    )

Tokenizing:   0%|          | 0/869 [00:00<?, ? examples/s]

Chamado tokenize len(queries_documents): 869
CPU times: user 1.27 s, sys: 49.3 ms, total: 1.31 s
Wall time: 1.14 s


In [49]:
%%time
# train_dataset.set_transform(tokenize)
train_dataset = train_dataset.map(
        tokenize,
        remove_columns=('query', 'text', 'label'),
        batched=True,
        desc='Tokenizing',
    )

Tokenizing:   0%|          | 0/85935 [00:00<?, ? examples/s]

Chamado tokenize len(queries_documents): 1000
Chamado tokenize len(queries_documents): 1000
Chamado tokenize len(queries_documents): 1000
Chamado tokenize len(queries_documents): 1000
Chamado tokenize len(queries_documents): 1000
Chamado tokenize len(queries_documents): 1000
Chamado tokenize len(queries_documents): 1000
Chamado tokenize len(queries_documents): 1000
Chamado tokenize len(queries_documents): 1000
Chamado tokenize len(queries_documents): 1000
Chamado tokenize len(queries_documents): 1000
Chamado tokenize len(queries_documents): 1000
Chamado tokenize len(queries_documents): 1000
Chamado tokenize len(queries_documents): 1000
Chamado tokenize len(queries_documents): 1000
Chamado tokenize len(queries_documents): 1000
Chamado tokenize len(queries_documents): 1000
Chamado tokenize len(queries_documents): 1000
Chamado tokenize len(queries_documents): 1000
Chamado tokenize len(queries_documents): 1000
Chamado tokenize len(queries_documents): 1000
Chamado tokenize len(queries_docum

In [50]:
print(valid_dataset[0])

{'input_ids': [2094, 540, 46, 149, 31, 2936, 13601, 7, 3217, 4, 657, 6275, 20, 10483, 4, 877, 1833, 4, 856, 48, 18522, 4, 2049, 12369, 8, 4, 3221, 4, 597, 4, 4192, 5, 745, 797, 1806, 46, 28, 762, 21, 15, 22728, 4342, 64, 5906, 4342, 87, 3269, 46, 15, 2091, 12884, 34, 1152, 7381, 8, 13589, 48, 3221, 1906, 11, 516, 3, 4, 237, 18, 42, 7415, 848, 8403, 501, 11, 1610, 1405, 4, 17318, 6, 10475, 6, 24, 8426, 6982, 73, 64, 5906, 4342, 21, 17, 14757, 4, 46, 15, 22728, 4, 4192, 64, 5906, 4342, 87, 762, 5993, 46, 15, 323, 252, 1308, 4, 20517, 49, 15, 5695, 61, 797, 93, 4, 597, 8, 1077, 49, 15, 20888, 4, 6205, 49, 15, 2950, 485, 2424, 4, 385, 1977, 49, 15, 2950, 485, 2424, 4, 1977, 8, 4624, 49, 15, 617, 10245, 49, 15, 175, 1166, 1450, 49, 15, 2091, 608, 2424, 8222, 49, 15, 2091, 608, 2424, 2966, 14, 9276, 58, 49, 15, 2091, 2154, 33, 1346, 1450, 49, 15, 2091, 405, 146, 24, 2950, 485, 2424, 13688, 15, 165, 254, 157, 6210, 4, 4, 6, 131, 2758, 555, 49, 15, 1968, 1404, 3992, 4, 6, 2114, 1091, 49, 15, 1

# Train

## setup

In [51]:
PATH_TRAIN_MODEL_LOCAL

'/content/drive/MyDrive/treinamento/202301_IA368DD/indir/train/monot5'

In [52]:
num_step_alert = 200
training_args = Seq2SeqTrainingArguments(output_dir=PATH_TRAIN_MODEL_LOCAL)
# Needed to make the Trainer work with an on-the-fly transformation on the dataset
# training_args.remove_unused_columns = False
training_args.output_dir = PATH_TRAIN_MODEL_LOCAL
training_args.warmup_steps=200 # Alterar!
training_args.num_train_epochs=3.0 # Alterar!
training_args.logging_steps=num_step_alert # Alterar!
training_args.save_strategy="steps"
training_args.save_steps=num_step_alert
training_args.save_total_limit=10
training_args.learning_rate=5e-5
training_args.per_device_train_batch_size=8 # t4: 8, a100-40: 32
training_args.gradient_accumulation_steps=4 # t4: 4, a100-40: 2
#training_args._n_gpu = 1
training_args.bf16 = True # se for usar a100, 3090, 4090 -> usar
training_args.ignore_data_skip = True
training_args.load_best_model_at_end = True
training_args.evaluation_strategy='steps'
training_args.eval_steps=num_step_alert
training_args.do_eval = True
# training_args.optim='adamw_hf' #default
training_args.gradient_checkpointing = False # True
# se precisar economizar gpu
# training_args.optim='adamw_bnb_8bit'
# training_args.gradient_checkpointing = True
# training_args.report_to="none"

In [53]:
!ls

drive  sample_data


In [54]:
print(training_args)

Seq2SeqTrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=True,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_backend=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=200,
evaluation_strategy=steps,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'fsdp_min_num_params': 0, 'xla': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
generation_config=None,
generation_max_length=None,
generation_num_beams=None,
gradient_accumulation_steps=4,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backen

In [55]:
# from transformers.integrations import NeptuneCallback
# rastro_neptune = NeptuneRastroRun(hparam, parm_lista_tag= tag_contexto_rastro)
# neptune_callback = NeptuneCallback(run=rastro_neptune)

In [56]:
%%time
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)


CPU times: user 2.51 s, sys: 1.03 s, total: 3.54 s
Wall time: 8.58 s


In [57]:
trainer_cls = Seq2SeqTrainer
data_collator = DataCollatorForSeq2Seq(tokenizer)

In [58]:
# Limpa o cache da memória da GPU
# del trainer
torch.cuda.empty_cache()

## Train

In [ ]:
raise Exception('Parar aqui reinício!')

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
Exception: Parar aqui reinício!

In [59]:
trainer = trainer_cls(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [ ]:
%%time
train_metrics = trainer.train(resume_from_checkpoint=True)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss


In [ ]:
print(train_metrics )

EXECUÇÕES ANTERIORES

Não sei se a mudança do batch size (32x2) 64 para 32 (8x4) impactou passar por dados duas vezes!!!

In [ ]:
%%time
train_metrics = trainer.train(resume_from_checkpoint=True)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
1000,0.169300,0.169204
1200,0.150800,0.164846
1400,0.147200,0.157921
1600,0.137600,0.158428
1800,0.148400,0.147066
2000,0.141600,0.143860
2200,0.137800,0.135446


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /usr/local/lib/python3.10/dist-packages/IPython/core/magics/execution.py:1335 in time            │
│                                                                                                  │
│   1332 │   │   else:                                                                             │
│   1333 │   │   │   st = clock2()                                                                 │
│   1334 │   │   │   try:                                                                          │
│ ❱ 1335 │   │   │   │   exec(code, glob, local_ns)                                                │
│   1336 │   │   │   │   out=None                                                                  │
│   1337 │   │   │   │   # multi-line %%time case                                                  │
│   1338 │   │   │   │   if expr_val is not None:                                                  │
│ in <module>:1                                                                                    │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:1645 in train                    │
│                                                                                                  │
│   1642 │   │   inner_training_loop = find_executable_batch_size(                                 │
│   1643 │   │   │   self._inner_training_loop, self._train_batch_size, args.auto_find_batch_size  │
│   1644 │   │   )                                                                                 │
│ ❱ 1645 │   │   return inner_training_loop(                                                       │
│   1646 │   │   │   args=args,                                                                    │
│   1647 │   │   │   resume_from_checkpoint=resume_from_checkpoint,                                │
│   1648 │   │   │   trial=trial,                                                                  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:2020 in _inner_training_loop     │
│                                                                                                  │
│   2017 │   │   │   │   │   self.state.epoch = epoch + (step + 1 + steps_skipped) / steps_in_epo  │
│   2018 │   │   │   │   │   self.control = self.callback_handler.on_step_end(args, self.state, s  │
│   2019 │   │   │   │   │                                                                         │
│ ❱ 2020 │   │   │   │   │   self._maybe_log_save_evaluate(tr_loss, model, trial, epoch, ignore_k  │
│   2021 │   │   │   │   else:                                                                     │
│   2022 │   │   │   │   │   self.control = self.callback_handler.on_substep_end(args, self.state  │
│   2023                                                                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:2307 in _maybe_log_save_evaluate │
│                                                                                                  │
│   2304 │   │   │   self._globalstep_last_logged = self.state.global_step                         │
│   2305 │   │   │   self.store_flos()                                                             │
│   2306 │   │   │                                                                                 │
│ ❱ 2307 │   │   │   self.log(logs)                                                                │
│   2308 │   │                                                                                     │
│   2309 │   │   metrics = None                              

In [62]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [63]:
train_metrics = trainer.train(resume_from_checkpoint=True)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:1645 in train                    │
│                                                                                                  │
│   1642 │   │   inner_training_loop = find_executable_batch_size(                                 │
│   1643 │   │   │   self._inner_training_loop, self._train_batch_size, args.auto_find_batch_size  │
│   1644 │   │   )                                                                                 │
│ ❱ 1645 │   │   return inner_training_loop(                                                       │
│   1646 │   │   │   args=args,                                                                    │
│   1647 │   │   │   resume_from_checkpoint=resume_from_checkpoint,                                │
│   1648 │   │   │   trial=trial,                                                                  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:1862 in _inner_training_loop     │
│                                                                                                  │
│   1859 │   │   self._globalstep_last_logged = self.state.global_step                             │
│   1860 │   │   model.zero_grad()                                                                 │
│   1861 │   │                                                                                     │
│ ❱ 1862 │   │   self.control = self.callback_handler.on_train_begin(args, self.state, self.contr  │
│   1863 │   │                                                                                     │
│   1864 │   │   # Skip the first epochs_trained epochs to get the random state of the dataloader  │
│   1865 │   │   if not args.ignore_data_skip:                                                     │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer_callback.py:353 in on_train_begin   │
│                                                                                                  │
│   350 │                                                                                          │
│   351 │   def on_train_begin(self, args: TrainingArguments, state: TrainerState, control: Trai   │
│   352 │   │   control.should_training_stop = False                                               │
│ ❱ 353 │   │   return self.call_event("on_train_begin", args, state, control)                     │
│   354 │                                                                                          │
│   355 │   def on_train_end(self, args: TrainingArguments, state: TrainerState, control: Traine   │
│   356 │   │   return self.call_event("on_train_end", args, state, control)                       │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer_callback.py:397 in call_event       │
│                                                                                                  │
│   394 │                                                                                          │
│   395 │   def call_event(self, event, args, state, control, **kwargs):                           │
│   396 │   │   for callback in self.callbacks:                                                    │
│ ❱ 397 │   │   │   result = getattr(callback, event)(                                             │
│   398 │   │   │   │   args,                                

Abaixo execucao lim50: a100/40gb (erro mount drive)

In [ ]:
%%time
train_metrics = trainer.train(resume_from_checkpoint=False)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
100,0.654600,0.284098
200,0.249600,0.231188
300,0.215400,0.206089
400,0.197600,0.203279
500,0.196000,0.189962
600,0.190700,0.180177
700,0.176600,0.174756
800,0.170100,0.169825


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /usr/local/lib/python3.10/dist-packages/IPython/core/magics/execution.py:1335 in time            │
│                                                                                                  │
│   1332 │   │   else:                                                                             │
│   1333 │   │   │   st = clock2()                                                                 │
│   1334 │   │   │   try:                                                                          │
│ ❱ 1335 │   │   │   │   exec(code, glob, local_ns)                                                │
│   1336 │   │   │   │   out=None                                                                  │
│   1337 │   │   │   │   # multi-line %%time case                                                  │
│   1338 │   │   │   │   if expr_val is not None:                                                  │
│ in <module>:1                                                                                    │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:1645 in train                    │
│                                                                                                  │
│   1642 │   │   inner_training_loop = find_executable_batch_size(                                 │
│   1643 │   │   │   self._inner_training_loop, self._train_batch_size, args.auto_find_batch_size  │
│   1644 │   │   )                                                                                 │
│ ❱ 1645 │   │   return inner_training_loop(                                                       │
│   1646 │   │   │   args=args,                                                                    │
│   1647 │   │   │   resume_from_checkpoint=resume_from_checkpoint,                                │
│   1648 │   │   │   trial=trial,                                                                  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:2020 in _inner_training_loop     │
│                                                                                                  │
│   2017 │   │   │   │   │   self.state.epoch = epoch + (step + 1 + steps_skipped) / steps_in_epo  │
│   2018 │   │   │   │   │   self.control = self.callback_handler.on_step_end(args, self.state, s  │
│   2019 │   │   │   │   │                                                                         │
│ ❱ 2020 │   │   │   │   │   self._maybe_log_save_evaluate(tr_loss, model, trial, epoch, ignore_k  │
│   2021 │   │   │   │   else:                                                                     │
│   2022 │   │   │   │   │   self.control = self.callback_handler.on_substep_end(args, self.state  │
│   2023                                                                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:2307 in _maybe_log_save_evaluate │
│                                                                                                  │
│   2304 │   │   │   self._globalstep_last_logged = self.state.global_step                         │
│   2305 │   │   │   self.store_flos()                                                             │
│   2306 │   │   │                                                                                 │
│ ❱ 2307 │   │   │   self.log(logs)                                                                │
│   2308 │   │                                                                                     │
│   2309 │   │   metrics = None                              

In [ ]:
train_metrics = trainer.train(resume_from_checkpoint=True)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:1645 in train                    │
│                                                                                                  │
│   1642 │   │   inner_training_loop = find_executable_batch_size(                                 │
│   1643 │   │   │   self._inner_training_loop, self._train_batch_size, args.auto_find_batch_size  │
│   1644 │   │   )                                                                                 │
│ ❱ 1645 │   │   return inner_training_loop(                                                       │
│   1646 │   │   │   args=args,                                                                    │
│   1647 │   │   │   resume_from_checkpoint=resume_from_checkpoint,                                │
│   1648 │   │   │   trial=trial,                                                                  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:1862 in _inner_training_loop     │
│                                                                                                  │
│   1859 │   │   self._globalstep_last_logged = self.state.global_step                             │
│   1860 │   │   model.zero_grad()                                                                 │
│   1861 │   │                                                                                     │
│ ❱ 1862 │   │   self.control = self.callback_handler.on_train_begin(args, self.state, self.contr  │
│   1863 │   │                                                                                     │
│   1864 │   │   # Skip the first epochs_trained epochs to get the random state of the dataloader  │
│   1865 │   │   if not args.ignore_data_skip:                                                     │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer_callback.py:353 in on_train_begin   │
│                                                                                                  │
│   350 │                                                                                          │
│   351 │   def on_train_begin(self, args: TrainingArguments, state: TrainerState, control: Trai   │
│   352 │   │   control.should_training_stop = False                                               │
│ ❱ 353 │   │   return self.call_event("on_train_begin", args, state, control)                     │
│   354 │                                                                                          │
│   355 │   def on_train_end(self, args: TrainingArguments, state: TrainerState, control: Traine   │
│   356 │   │   return self.call_event("on_train_end", args, state, control)                       │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer_callback.py:397 in call_event       │
│                                                                                                  │
│   394 │                                                                                          │
│   395 │   def call_event(self, event, args, state, control, **kwargs):                           │
│   396 │   │   for callback in self.callbacks:                                                    │
│ ❱ 397 │   │   │   result = getattr(callback, event)(                                             │
│   398 │   │   │   │   args,                                

In [ ]:
%%time
train_metrics = trainer.train(resume_from_checkpoint=False)

Step,Training Loss,Validation Loss
100,0.667000,0.281231
200,0.257300,0.202263
300,0.225900,0.183011
400,0.208300,0.171449
500,0.196900,0.155438
600,0.181600,0.146543
700,0.178900,0.142439
800,0.173300,0.144331
900,0.173400,0.134399
1000,0.172500,0.128392


CPU times: user 1h 58min 40s, sys: 6min 29s, total: 2h 5min 10s
Wall time: 2h 6min 24s


In [ ]:
train_metrics = trainer.train(resume_from_checkpoint=True)

Step,Training Loss,Validation Loss


Exception in thread Thread-13:
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/local/lib/python3.10/dist-packages/tensorboard/summary/writer/event_file_writer.py", line 244, in run
    self._run()
  File "/usr/local/lib/python3.10/dist-packages/tensorboard/summary/writer/event_file_writer.py", line 289, in _run
    self._record_writer.flush()
  File "/usr/local/lib/python3.10/dist-packages/tensorboard/summary/writer/record_writer.py", line 43, in flush
    self._writer.flush()
  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/lib/io/file_io.py", line 221, in flush
    self._writable_file.flush()
tensorflow.python.framework.errors_impl.FailedPreconditionError: /content/drive/MyDrive/treinamento/202301_IA368DD/indir/train/monot5/runs/Jun25_17-17-10_4215e4daa1aa/events.out.tfevents.1687713483.4215e4daa1aa.2006.2; Transport endpoint is not connected


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:1645 in train                    │
│                                                                                                  │
│   1642 │   │   inner_training_loop = find_executable_batch_size(                                 │
│   1643 │   │   │   self._inner_training_loop, self._train_batch_size, args.auto_find_batch_size  │
│   1644 │   │   )                                                                                 │
│ ❱ 1645 │   │   return inner_training_loop(                                                       │
│   1646 │   │   │   args=args,                                                                    │
│   1647 │   │   │   resume_from_checkpoint=resume_from_checkpoint,                                │
│   1648 │   │   │   trial=trial,                                                                  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:2020 in _inner_training_loop     │
│                                                                                                  │
│   2017 │   │   │   │   │   self.state.epoch = epoch + (step + 1 + steps_skipped) / steps_in_epo  │
│   2018 │   │   │   │   │   self.control = self.callback_handler.on_step_end(args, self.state, s  │
│   2019 │   │   │   │   │                                                                         │
│ ❱ 2020 │   │   │   │   │   self._maybe_log_save_evaluate(tr_loss, model, trial, epoch, ignore_k  │
│   2021 │   │   │   │   else:                                                                     │
│   2022 │   │   │   │   │   self.control = self.callback_handler.on_substep_end(args, self.state  │
│   2023                                                                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:2307 in _maybe_log_save_evaluate │
│                                                                                                  │
│   2304 │   │   │   self._globalstep_last_logged = self.state.global_step                         │
│   2305 │   │   │   self.store_flos()                                                             │
│   2306 │   │   │                                                                                 │
│ ❱ 2307 │   │   │   self.log(logs)                                                                │
│   2308 │   │                                                                                     │
│   2309 │   │   metrics = None                                                                    │
│   2310 │   │   if self.control.should_evaluate:                                                  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:2672 in log                      │
│                                                                                                  │
│   2669 │   │                                                                                     │
│   2670 │   │   output = {**logs, **{"step": self.state.global_step}}                             │
│   2671 │   │   self.state.log_history.append(output)                                             │
│ ❱ 2672 │   │   self.control = self.callback_handler.on_log(self.args, self.state, self.control,  │
│   2673 │                                                   

In [ ]:
train_metrics

TrainOutput(global_step=1730, training_loss=0.20708703939625295, metrics={'train_runtime': 7584.6743, 'train_samples_per_second': 14.6, 'train_steps_per_second': 0.228, 'total_flos': 6.74238176428032e+16, 'train_loss': 0.20708703939625295, 'epoch': 1.0})

In [ ]:
huggingface-cli login

╭──────────────────────────────────────────────────────────────────────────────────────────────────╮
│ huggingface-cli login                                                                            │
│                 ▲                                                                                │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
SyntaxError: invalid syntax

In [ ]:
pip install huggingface_hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from huggingface_hub import notebook_login

In [ ]:
notebook_login()


In [ ]:
trainer.push_to_hub(
    model_id="ptt5-base-pt-msmarco-100k-v2-indir-lim100",
    repo_name="marcusborela"
    # use_auth_token=""
)